<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Chapter 2: Working with Text Data

이 노트북에서 사용되는 패키지는 다음과 같습니다.:

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1
tiktoken version: 0.7.0


- 이 장에서는 LLM에 사용할 입력 데이터를 "준비"하기 위한 데이터 준비 및 샘플링에 대해 다룹니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="1000px">

## 2.1 Understanding word embeddings

- No code in this section

- 임베딩에는 여러 가지 형태가 있지만, 이 책에서는 텍스트 임베딩에 초점을 맞춥니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="1000px">

- LLM은 고차원 공간(즉, 수천 개의 차원)의 임베딩을 사용합니다.
- 인간은 1, 2, 3차원으로 생각하기 때문에 이러한 고차원 공간을 시각화할 수 없으므로 아래 그림은 2차원 임베딩 공간을 나타냅니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="600px">

## 2.2 Tokenizing text

- In this section, we tokenize text, which means breaking text into smaller units, such as individual words and punctuation characters

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="600px">

- Load raw text we want to work with
- [The Verdict by Edith Wharton](https://en.wikisource.org/wiki/The_Verdict) is a public domain short story

In [2]:
import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)


# The book originally used the following code below
# However, urllib uses older protocol settings that
# can cause problems for some readers using a VPN.
# The `requests` version above is more robust
# in that regard.

"""
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)
"""

<br>

---

<br>

#### Troubleshooting SSL certificate errors

- Some readers reported seeing ssl.SSLCertVerificationError: `SSL: CERTIFICATE_VERIFY_FAILED` when running `urllib.request.urlretrieve` in VSCode or Jupyter. 
- This usually means Python's certificate bundle is outdated.


**Fixes**

- Use Python ≥ 3.9; you can check your Python version by executing the following code:
```python
import sys
print(sys.__version__)
```
- Upgrade the cert bundle:
  - pip: `pip install --upgrade certifi`
  - uv: `uv pip install --upgrade certifi`
- Restart the Jupyter kernel after upgrading.
- If you still encounter an `ssl.SSLCertVerificationError` when executing the previous code cell, please see the discussion at [more information here on GitHub](https://github.com/rasbt/LLMs-from-scratch/pull/403)

<br>

---

<br>

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 목표는 이 텍스트를 토큰화하고 LLM에 삽입하는 것입니다.
- 먼저 간단한 샘플 텍스트를 기반으로 간단한 토크나이저를 개발하고, 나중에 이를 위의 텍스트에 적용해 보겠습니다.
- 다음 정규 표현식은 공백을 기준으로 텍스트를 분할합니다.

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 공백뿐만 아니라 쉼표와 마침표로도 분할하고 싶으므로, 정규 표현식을 수정하여 그렇게 하도록 하겠습니다.

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 보시다시피, 이렇게 하면 빈 문자열이 생성되므로 이를 제거해 보겠습니다.

In [6]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 꽤 괜찮아 보이지만, 마침표, 물음표 등 다른 종류의 문장 부호도 처리해 봅시다.

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 꽤 괜찮네요. 이제 이 토큰화를 원시 텍스트에 적용할 준비가 되었습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="700px">

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 토큰의 총 개수를 계산해 봅시다.

In [9]:
print(len(preprocessed))

4690


## 2.3 Converting tokens into token IDs

- 다음으로, 텍스트 토큰을 나중에 임베딩 레이어를 통해 처리할 수 있는 토큰 ID로 변환합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="1000px">

- 이러한 토큰들을 이용하여 이제 모든 고유 토큰으로 구성된 어휘집을 구축할 수 있습니다.

In [10]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [11]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 아래는 이 어휘집의 첫 50개 항목입니다.

In [12]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 아래에서는 적은 어휘를 사용하여 짧은 샘플 텍스트를 토큰화하는 방법을 보여줍니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="1000px">

- 이제 이 모든 것을 토크나이저 클래스로 통합해 보겠습니다.

In [13]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 함수는 텍스트를 토큰 ID로 변환합니다.
- `decode` 함수는 토큰 ID를 다시 텍스트로 변환합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="1000px">


- 토크나이저를 사용하여 텍스트를 정수로 인코딩(토큰화)할 수 있습니다.
- 이렇게 생성된 정수는 나중에 LLM의 입력으로 사용될 수 있습니다.

In [14]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 정수를 다시 텍스트로 해독할 수 있습니다.

In [15]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [16]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 Adding special context tokens


- 알 수 없는 단어를 표시하거나 텍스트의 끝을 나타내기 위해 몇 가지 "특수" 토큰을 추가하는 것이 유용합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="1000px">

- 일부 토크나이저는 LLM에 추가적인 문맥 정보를 제공하기 위해 특수 토큰을 사용합니다.
- 이러한 특수 토큰의 예는 다음과 같습니다.

- `[BOS]` (시퀀스 시작): 텍스트의 시작을 나타냅니다.

- `[EOS]` (시퀀스 끝): 텍스트의 끝을 나타냅니다. (이 토큰은 일반적으로 서로 관련 없는 여러 텍스트를 연결하는 데 사용됩니다. 예를 들어, 서로 다른 위키백과 문서 두 개 또는 서로 다른 책 두 권 등을 연결할 때 사용됩니다.)

- `[PAD]` (패딩): 배치 크기가 1보다 큰 LLM을 학습할 때 사용됩니다. (길이가 다른 여러 텍스트를 포함할 수 있으며, 패딩 토큰을 사용하여 짧은 텍스트를 가장 긴 텍스트 길이로 채워 모든 텍스트의 길이가 같도록 합니다.)
- `[UNK]`: 어휘에 포함되지 않은 단어를 나타냅니다.

- GPT-2는 위에서 언급한 토큰이 필요하지 않으며, 복잡성을 줄이기 위해 `<|endoftext|>` 토큰만 사용합니다.
- `<|endoftext|>`는 위에서 언급한 `[EOS]` 토큰과 유사합니다.
- GPT는 패딩을 위해 `<|endoftext|>`를 사용합니다(일반적으로 배치 입력으로 학습할 때 마스크를 사용하므로 패딩된 토큰에는 어차피 주의를 기울이지 않기 때문에 이러한 토큰이 무엇인지는 중요하지 않습니다).
- GPT-2는 어휘 외 단어에 대해 `<UNK>` 토큰을 사용하지 않습니다. 대신 GPT-2는 바이트 쌍 인코딩(BPE) 토크나이저를 사용하여 단어를 하위 단어 단위로 분해하는데, 이에 대해서는 나중에 자세히 설명하겠습니다.

- We use the `<|endoftext|>` tokens between two independent sources of text:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="1000px">


- 다음 텍스트를 토큰화하면 어떻게 되는지 살펴보겠습니다.

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 위 코드는 "Hello"라는 단어가 어휘에 포함되어 있지 않기 때문에 오류를 발생시킵니다.
- 이러한 경우를 처리하기 위해 알 수 없는 단어를 나타내는 `<|unk|>`와 같은 특수 토큰을 어휘에 추가할 수 있습니다.
- 어휘를 확장하는 김에 GPT-2 학습에서 텍스트의 끝을 나타내는 데 사용되는 `<|endoftext|>`라는 토큰도 추가해 보겠습니다. (이 토큰은 학습 데이터셋이 여러 개의 기사, 책 등으로 구성된 경우처럼 연결된 텍스트 사이에도 사용됩니다.)

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 또한 토크나이저가 새로운 `<unk>` 토큰을 언제 어떻게 사용해야 하는지 알 수 있도록 토크나이저를 적절히 조정해야 합니다.

In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

수정된 토크나이저를 사용하여 텍스트를 토큰화해 보겠습니다.

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BytePair encoding

- GPT-2는 토크나이저로 바이트페어 인코딩(BPE)을 사용했습니다.
- 이를 통해 모델은 미리 정의된 어휘에 없는 단어를 더 작은 하위 단어 단위 또는 개별 문자로 분해할 수 있으므로 어휘 외 단어를 처리할 수 있습니다.
- 예를 들어, GPT-2의 어휘에 "unfamiliarword"라는 단어가 없는 경우, 학습된 BPE 병합에 따라 ["unfam", "iliar", "word"] 또는 다른 하위 단어 분해로 토큰화할 수 있습니다.
- 원본 BPE 토크나이저는 다음 링크에서 확인할 수 있습니다. [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 이 장에서는 OpenAI의 오픈 소스 [tiktoken](https://github.com/openai/tiktoken)의 BPE 토크나이저를 사용합니다. 이 라이브러리는 핵심 알고리즘을 Rust로 구현하여 연산 성능을 향상시킵니다.
- [./bytepair_encoder](../02_bonus_bytepair-encoder)에 이 두 구현을 나란히 비교하는 노트북을 만들었습니다. (샘플 텍스트에서 tiktoken이 약 5배 더 빨랐습니다.)

In [25]:
# pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.



- BPE 토크나이저는 알 수 없는 단어를 하위 단어와 개별 문자로 분해합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="600px">

## 2.6 Data sampling with a sliding window


- 우리는 LLM이 한 번에 한 단어씩 생성하도록 학습시키므로, 시퀀스에서 다음 단어가 예측 대상 단어를 나타내도록 학습 데이터를 준비해야 합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="800px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 각 텍스트 덩어리에 대해 입력값과 목표값을 구하고자 합니다.
- 모델이 다음 단어를 예측하도록 하려면 목표값은 입력값을 오른쪽으로 한 칸 이동시킨 값입니다.

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 하나씩 살펴보면 예측 결과는 다음과 같습니다.

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 다음 단어 예측은 어텐션 메커니즘을 다룬 후 다음 장에서 자세히 살펴보겠습니다.
- 지금은 입력 데이터셋을 순회하며 입력값과 목표값을 1씩 이동시켜 반환하는 간단한 데이터 로더를 구현했습니다.

- Install and import PyTorch (see Appendix A for installation tips)

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1


- 우리는 슬라이딩 윈도우 방식을 사용하여 위치를 +1씩 변경합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="1000px">


- 입력 텍스트 데이터셋에서 청크를 추출하는 데이터셋과 데이터로더를 생성합니다.

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- Let's test the dataloader with a batch size of 1 for an LLM with a context size of 4:

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 컨텍스트 크기가 4인 LLM에 대해 배치 크기를 1로 설정하여 데이터로더를 테스트해 보겠습니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="1000px">


- 일괄 출력도 생성할 수 있습니다.
- 여기서는 배치 간 중복을 방지하기 위해 스트라이드를 늘립니다. 중복이 많을수록 과적합이 증가할 수 있기 때문입니다.

In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating token embeddings

- 데이터는 이미 LLM을 사용하기에 거의 준비가 되었습니다.
- 마지막으로 임베딩 레이어를 사용하여 토큰을 연속 벡터 표현으로 변환해 보겠습니다.
- 일반적으로 이러한 임베딩 레이어는 LLM 자체의 일부이며 모델 학습 중에 업데이트(학습)됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="800px">

- 토큰화 후 입력 ID가 각각 2, 3, 5, 1인 다음 네 가지 입력 예제가 있다고 가정해 보겠습니다.

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])


- 이해를 쉽게 하기 위해, 단어 수가 6개에 불과한 작은 어휘를 가지고 있고 크기가 3인 임베딩을 생성한다고 가정해 보겠습니다.

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 이렇게 하면 6x3 크기의 가중치 행렬이 생성됩니다.|

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 원핫 인코딩에 익숙한 사람이라면 위의 임베딩 레이어 접근 방식은 본질적으로 완전 연결 레이어에서 원핫 인코딩 후 행렬 곱셈을 수행하는 보다 효율적인 구현 방식일 뿐이며, 이는 [./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)의 보충 코드에 설명되어 있습니다.
- 임베딩 레이어는 원핫 인코딩 및 행렬 곱셈 방식과 동일한 결과를 제공하는 보다 효율적인 구현 방식이기 때문에 역전파를 통해 최적화할 수 있는 신경망 레이어로 볼 수 있습니다.

- ID가 3인 토큰을 3차원 벡터로 변환하려면 다음과 같이 합니다.


In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)



- 위 내용은 `embedding_layer` 가중치 행렬의 네 번째 행입니다.
- 위의 네 개의 `input_ids` 값을 모두 임베딩하려면 다음과 같이 합니다.

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 임베딩 레이어는 기본적으로 조회 작업입니다.


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="1000px">

- **임베딩 레이어와 일반 선형 레이어를 비교하는 보너스 콘텐츠에 관심이 있으실 수도 있습니다: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 Encoding word positions

- 임베딩 레이어는 입력 시퀀스 내 위치에 관계없이 ID를 동일한 벡터 표현으로 변환합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="800px">


- 위치 임베딩은 토큰 임베딩 벡터와 결합되어 대규모 언어 모델의 입력 임베딩을 구성합니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="1000px">


- BytePair 인코더의 어휘 크기는 50,257입니다.
- 입력 토큰을 256차원 벡터 표현으로 인코딩한다고 가정해 보겠습니다.

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 데이터로더에서 데이터를 샘플링하면 각 배치에 있는 토큰을 256차원 벡터에 임베딩합니다.
- 배치 크기가 8이고 각 배치에 4개의 토큰이 있는 경우, 결과적으로 8 x 4 x 256 크기의 텐서가 생성됩니다.

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2는 절대 위치 임베딩을 사용하므로 임베딩 레이어를 하나 더 생성합니다.


In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# uncomment & execute the following line to see how the embedding layer weights look like
# print(pos_embedding_layer.weight)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(pos_embeddings)

torch.Size([4, 256])



LLM에서 사용되는 입력 임베딩을 생성하려면 토큰과 위치 임베딩을 추가하기만 하면 됩니다.

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# uncomment & execute the following line to see how the embeddings look like
# print(input_embeddings)

torch.Size([8, 4, 256])



- 입력 처리 워크플로의 초기 단계에서 입력 텍스트는 개별 토큰으로 분할됩니다.
- 이러한 분할 후, 해당 토큰들은 미리 정의된 어휘집을 기반으로 토큰 ID로 변환됩니다.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="800x">

# Summary and takeaways

See the [./dataloader.ipynb](./dataloader.ipynb) code notebook, which is a concise version of the data loader that we implemented in this chapter and will need for training the GPT model in upcoming chapters.

See [./exercise-solutions.ipynb](./exercise-solutions.ipynb) for the exercise solutions.

See the [Byte Pair Encoding (BPE) Tokenizer From Scratch](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) notebook if you are interested in learning how the GPT-2 tokenizer can be implemented and trained from scratch.